In [21]:
import pandas as pd
import numpy as np
import random 

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, mean_absolute_error

import xgboost
import numerapi
NAPI = numerapi.NumerAPI(verbosity="info")
import random as rn
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
from bayes_opt import BayesianOptimization
import os
import uuid
import pickle

from utils import *
from poly_features import *
from xgboost_feval import *
DIR = "data"

In [69]:
train_static.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_charisma_skew,feature_dexterity_mean,feature_dexterity_std,feature_dexterity_skew,feature_strength_mean,feature_strength_std,feature_strength_skew,feature_constitution_mean,feature_constitution_std,feature_constitution_skew
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,...,-0.004783,0.696429,0.200446,-0.607620,0.480263,0.292829,-0.372064,0.427632,0.275720,0.276155
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,...,-0.021737,0.267857,0.249312,0.382267,0.407895,0.309866,0.220625,0.644737,0.334080,-0.794938
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,...,-0.600709,0.446429,0.355973,0.624266,0.203947,0.289777,1.258705,0.418860,0.331755,0.179824
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,...,-0.238108,0.232143,0.118658,-0.308327,0.394737,0.332027,0.332149,0.429825,0.321619,0.541559
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,...,-0.329475,0.500000,0.138675,0.000000,0.342105,0.262660,1.548723,0.508772,0.394836,0.092206


In [70]:
train_static['era'].describe()

count     501808
unique       120
top        era55
freq        4893
Name: era, dtype: object

In [74]:
val_static['target'].describe()

count    137779.000000
mean          0.500015
std           0.223491
min           0.000000
25%           0.500000
50%           0.500000
75%           0.500000
max           1.000000
Name: target, dtype: float64

In [73]:
tournament_static['era'].describe()

count     1682185
unique        332
top          eraX
freq         5443
Name: era, dtype: object

In [76]:
grouped = train_static.groupby('era')

for name, group in grouped:
    print(name)

era1
era10
era100
era101
era102
era103
era104
era105
era106
era107
era108
era109
era11
era110
era111
era112
era113
era114
era115
era116
era117
era118
era119
era12
era120
era13
era14
era15
era16
era17
era18
era19
era2
era20
era21
era22
era23
era24
era25
era26
era27
era28
era29
era3
era30
era31
era32
era33
era34
era35
era36
era37
era38
era39
era4
era40
era41
era42
era43
era44
era45
era46
era47
era48
era49
era5
era50
era51
era52
era53
era54
era55
era56
era57
era58
era59
era6
era60
era61
era62
era63
era64
era65
era66
era67
era68
era69
era7
era70
era71
era72
era73
era74
era75
era76
era77
era78
era79
era8
era80
era81
era82
era83
era84
era85
era86
era87
era88
era89
era9
era90
era91
era92
era93
era94
era95
era96
era97
era98
era99


In [23]:
# Download, unzip and load data

download_current_data(DIR)
train_static, val_static, tournament_static = load_data(DIR, reduce_memory=True)
features_list = generate_features_list(train_static)

data/numerai_dataset_257.zip: 100%|█████████▉| 397M/397M [03:04<00:00, 2.17MB/s] 2021-03-27 12:43:51,296 INFO numerapi.base_api: unzipping file...


Loading the data


data/numerai_dataset_257.zip: 397MB [03:20, 1.98MB/s]                           


In [24]:
def generate_submission(df, name="submission.csv"):
    by = pd.read_csv('data/numerai_dataset_'+str(NAPI.get_current_round())+'/example_predictions.csv')
    submission = pd.concat([by.drop(columns='prediction'), df['prediction']], axis=1)
    submission = submission.set_index('id')
    submission.to_csv(name)

In [25]:
# Group stats
train_with_group, val_with_group, tournament_with_group = get_group_stats(train_static), get_group_stats(val_static), get_group_stats(tournament_static)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [26]:
corr = train_with_group[features_list].corr()
tricorr = corr.where(~np.tril(np.ones(corr.shape)).astype(bool))
tricorr = tricorr.stack()
tricorr.describe()

count    47895.000000
mean         0.045188
std          0.150916
min         -0.855008
25%         -0.026995
50%          0.022809
75%          0.092002
max          0.968062
dtype: float64

In [27]:
from sklearn.decomposition import PCA
pca = PCA(0.95, svd_solver='full')
pca.fit(train_with_group[features_list])
res = pca.transform(train_with_group[features_list])
sum(pca.explained_variance_ratio_)

0.9505659995496654

In [103]:
train_with_group_pca = pca.transform(train_with_group[features_list])
val_with_group_pca = pca.transform(val_with_group[features_list])
tournament_with_group_pca = pca.transform(tournament_with_group[features_list])

In [104]:
train_with_group_pca_df = pd.DataFrame(train_with_group_pca)
val_with_group_pca_df = pd.DataFrame(val_with_group_pca)
tournament_with_group_pca_df = pd.DataFrame(tournament_with_group_pca)

In [105]:
pca_train = pd.concat([train_with_group.drop(columns=features_list).reset_index(),
                       train_with_group_pca_df.reset_index()], axis=1)
pca_val = pd.concat([val_with_group.drop(columns=features_list).reset_index(),
                     val_with_group_pca_df.reset_index()], axis=1)
pca_tournament = pd.concat([tournament_with_group.drop(columns=features_list).reset_index(),
                            tournament_with_group_pca_df.reset_index()], axis=1)

In [106]:
pca_train = pca_train.drop(columns=['index'])
pca_val = pca_val.drop(columns=['index'])
pca_tournament = pca_tournament.drop(columns=['index'])

In [107]:
submissions = []
X_tournament, y_tournament = clean_for_xgboost(pca_tournament)
dtournament = xgboost.DMatrix(X_tournament, y_tournament)
# model_name = 'boomkin'
# model_id = '6bd2c44b-38bb-4918-9f08-cc1e880bc6a5'
# model = load_model(f'models/{model_name}/model_{model_id}.pickle.dat')
pca_tournament.loc[:,"prediction"] = model.predict(dtournament)
submissions.append(pca_tournament)
generate_submission(pca_tournament, name=f'model_{model_name}_submission.csv')

In [33]:
def neutralize(series, by, proportion):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)
    exposures = np.hstack((exposures, np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))
    correction = proportion * (exposures.dot(np.linalg.lstsq(exposures, scores)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized

by = pd.read_csv('data/numerai_dataset_'+str(NAPI.get_current_round())+'/example_predictions.csv')
neut = pd.DataFrame({'prediction': neutralize(pca_tournament['prediction'], by['prediction'], 0.3)})

conc = pd.concat([by.drop(columns="prediction"),neut], axis=1)
conc.to_csv("neutralized_model_boomkin_submission.csv", index=False) # submission file

<ipython-input-33-e06180fd2d06>:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  correction = proportion * (exposures.dot(np.linalg.lstsq(exposures, scores)[0]))


In [42]:
sum(n < 0 for n in conc.prediction.values.flatten())
num = conc._get_numeric_data()
num[num < 0] = 0

0

In [53]:
def correlation_score(preds, dtrain):
    return "corr", np.corrcoef(preds, dtrain.get_label())[0,1]

In [ ]:
X_tournament, y_tournament = clean_for_xgboost(pca_tournament)
dtournament = xgboost.DMatrix(X_tournament, y_tournament)

In [100]:
def train_model(train, val, model=None):
    X_train, y_train = clean_for_xgboost(train)
    X_val, y_val = clean_for_xgboost(val)

    dtrain = xgboost.DMatrix(X_train, y_train)
    dtest = xgboost.DMatrix(X_val, y_val)

    param = {
        'colsample_bytree': 0.7334,
        'gamma': 1.412,
        'max_depth': 10,
        'min_child_weight': 18.26,
        'subsample': 1.0,
        'eta': 0.05,
        'objective': 'reg:squarederror',
        'eval_metric':'logloss',
        'verbosity': 0
    }
    evals_result = {}
    
    if model is not None:
        tmodel = xgboost.train(
            params=param,
            dtrain=dtrain,
#             feval=xgb_r2,
            num_boost_round=1000,
            evals=[(dtrain, 'train'), (dtest, 'test')],
            evals_result=evals_result,
            xgb_model=model,
            verbose_eval=10,
        )
    else:
        tmodel = xgboost.train(
            params=param,
            dtrain=dtrain,
#             feval=xgb_r2,
            num_boost_round=1000,
            evals=[(dtrain, 'train'), (dtest, 'test')],
            evals_result=evals_result,
            verbose_eval=10,
        )

#     hex_gen = uuid.uuid4()
#     print(f"Hex number associated with this is {hex_gen}")
#     save_model(model, f'model_{hex_gen}.pickle.dat')
    return tmodel

In [101]:
def train_test_dataframe(df):
    msk = np.random.rand(len(df)) < 0.8
    train = df[msk]
    test = df[~msk]
    return train, test

In [109]:
grouped = pca_train.groupby('era')
for name, group in grouped:
    train, test = train_test_dataframe(group)
    print("Current era", name)
    model = None
    model = train_model(train, test, model)

Current era era1
[1]	train-logloss:0.69314	test-logloss:0.69316
[2]	train-logloss:0.69314	test-logloss:0.69317
[3]	train-logloss:0.69314	test-logloss:0.69317
[4]	train-logloss:0.69314	test-logloss:0.69318
[5]	train-logloss:0.69314	test-logloss:0.69318
[6]	train-logloss:0.69314	test-logloss:0.69319
[7]	train-logloss:0.69314	test-logloss:0.69319
[8]	train-logloss:0.69314	test-logloss:0.69320
[9]	train-logloss:0.69314	test-logloss:0.69320
[11]	train-logloss:0.69279	test-logloss:0.69313
[12]	train-logloss:0.69279	test-logloss:0.69314
[13]	train-logloss:0.69279	test-logloss:0.69314
[14]	train-logloss:0.69279	test-logloss:0.69315
[15]	train-logloss:0.69279	test-logloss:0.69315
[16]	train-logloss:0.69279	test-logloss:0.69315
[17]	train-logloss:0.69279	test-logloss:0.69316
[18]	train-logloss:0.69279	test-logloss:0.69316
[19]	train-logloss:0.69279	test-logloss:0.69316
[21]	train-logloss:0.69279	test-logloss:0.69317
[22]	train-logloss:0.69279	test-logloss:0.69317
[23]	train-logloss:0.69279	test-

KeyboardInterrupt: 

In [108]:
save_model(model, f'model_boomkin_testing_logloss_1000.pickle.dat')

In [48]:
def hyp_xgb(max_depth, subsample, colsample_bytree, min_child_weight, gamma):
    params = {
      'eta': 0.05,
      'objective': 'reg:squarederror',
      'eval_metric':'logloss', # Optional --> Use eval_metric if you want to stop evaluation based on eval_metric 
      'verbosity': 0
    }
    params['max_depth'] = int(round(max_depth))
    params['subsample'] = max(min(subsample, 1), 0)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['min_child_weight'] = int(min_child_weight)
    params['gamma'] = max(gamma, 0)
    
    scores = xgboost.cv(params,
                    dtrain,
                    num_boost_round=10,
                    verbose_eval=False,
                    early_stopping_rounds=5,
                    feval=correlation_score,
                    maximize=True,
                    nfold=5)
    return  scores['test-corr-mean'].iloc[-1]
pds ={
  'min_child_weight':(14, 20),
  'gamma':(0, 5),
  'subsample':(0.5, 1),
  'colsample_bytree':(0.1, 1),
  'max_depth': (5, 10)
}

In [ ]:
X_train, y_train = clean_for_xgboost(pca_train)
dtrain = xgboost.DMatrix(X_train, y_train)

In [54]:
# Surrogate model
optimizer = BayesianOptimization(hyp_xgb, pds, random_state=101)                   
# Optimize
optimizer.maximize(init_points=5, n_iter=15)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------
|  1        |  0.02035  |  0.5648   |  2.853    |  5.142    |  15.03    |  0.8426   |
|  2        |  0.02446  |  0.8505   |  1.535    |  9.468    |  18.33    |  0.595    |
|  3        |  0.02291  |  0.5988   |  1.761    |  5.909    |  18.71    |  0.9827   |
|  4        |  0.01973  |  0.3091   |  0.4178   |  8.018    |  18.37    |  0.6381   |
|  5        |  0.01749  |  0.7168   |  2.589    |  5.242    |  14.83    |  0.5935   |
|  6        |  0.02388  |  0.7079   |  1.778    |  5.855    |  18.67    |  0.7764   |
|  7        |  0.02366  |  0.9361   |  1.604    |  9.371    |  18.53    |  0.6457   |
|  8        |  0.02189  |  0.8296   |  1.917    |  10.0     |  18.0     |  0.5314   |
|  9        |  0.0202   |  0.5885   |  2.221    |  7.087    |  16.12    |  0.5198   |
|  10       |  0.02263  |  0.9528   |  0.9688   |  9.5

In [ ]:
|  18       |  0.02606  |  0.7334   |  1.412    |  9.64     |  18.26    |  1.0      |
|  14       |  0.02553  |  0.5      |  1.431    |  9.719    |  18.5     |  0.865    |
|  2        |  0.02446  |  0.8505   |  1.535    |  9.468    |  18.33    |  0.595    |